# Test for GPU

Tests copied from the normal tests to be exectuded in a Jupyter notebook on the HPC by hand

In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `/p/tmp/maxgelbr/code/qg3.jl/test-cuda`


In [2]:
using QG3, CUDA

[ Info: Precompiling QG3 [d558103f-7907-4730-8f30-3d9252e5a318]


In [3]:
CUDA.functional()

true

In [4]:
CUDA.versioninfo()

CUDA runtime 11.4, artifact installation
CUDA driver 11.2
NVIDIA driver 460.106.0

CUDA libraries: 
- CUBLAS: 11.6.5
- CURAND: 10.2.5
- CUFFT: 10.5.2
- CUSOLVER: 11.2.0
- CUSPARSE: 11.6.0
- CUPTI: 14.0.0
- NVML: 11.0.0+460.106.0

Julia packages: 
- CUDA: 5.0.0
- CUDA_Driver_jll: 0.6.0+4
- CUDA_Runtime_jll: 0.9.2+3

Toolchain:
- Julia: 1.9.1
- LLVM: 14.0.6
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

1 device:
  0: Tesla V100-PCIE-32GB (sm_70, 31.745 GiB / 31.749 GiB available)


In [5]:
using CUDA.CUFFT, Test
import FFTW

In [6]:
A = CUDA.rand(100);
W = CUDA.rand(100);
V = CUDA.rand(102);

Ac = Array(A);

A2 = CUDA.rand(10,100);
W2 = CUDA.rand(10,100);

A3 = CUDA.rand(10,5,100);
W3 = CUDA.rand(10,5,100);

In [7]:
fft_plan = QG3.plan_r2r_AD(A, 1)
ifft_plan = QG3.plan_ir2r_AD(fft_plan * A, 100, 1)

Differentiable R2R wrapper of CUFFT complex-to-real backward plan for 51-element CuArray of ComplexF32

In [8]:
cpu_fft_plan = QG3.plan_r2r_AD(Ac, 1)

Differentiable R2R wrapper of FFTW r2r R2HC plan for 100-element array of Float32
(rdft-ct-dit/5
  (hc2hc-direct-5/4 "hf2_5"
    (rdft-r2hc-direct-r2c-5 "r2cf_5")
    (rdft-r2hc01-direct-r2c-5 "r2cfII_5"))
  (rdft-r2hc-direct-r2c-20-x5 "r2cf_20"))

In [10]:
@test Array((fft_plan * A)[1:50]) ≈ (cpu_fft_plan * Ac)[1:50] 
@test Array((fft_plan * A)[53:end-1]) ≈ (cpu_fft_plan * Ac)[end:-1:52] # reverse order in FFTW HC Format
@test (fft_plan \ (fft_plan * A)) ≈ (A * size(A,1))
@test ifft_plan * (fft_plan * A) ≈ (A * size(A,1))
@test ifft_plan \ (ifft_plan * (fft_plan * A)) ≈ ((fft_plan * A) * size(A,1))


Test Passed

In [11]:
ifft_plan * (fft_plan * A) ≈ (A * size(A,1))

true

basic_test_gpu.jl

In [6]:
# load forcing and model parameters

    using QG3, BenchmarkTools, OrdinaryDiffEq, JLD2

    S, qg3ppars, ψ_0, q_0 = QG3.load_precomputed_data()

    QG3.gpuoff()
    qg3p_cpu = QG3Model(qg3ppars)
    QG3.gpuon()

    S_gpu, qg3ppars_gpu, ψ_0_gpu, q_0_gpu = QG3.reorder_SH_gpu(S, qg3ppars), togpu(qg3ppars), QG3.reorder_SH_gpu(ψ_0, qg3ppars), QG3.reorder_SH_gpu(q_0, qg3ppars)

    qg3p_gpu = CUDA.@allowscalar QG3Model(qg3ppars_gpu)
    T = eltype(qg3p_gpu)
   

Float32

In [13]:
@test transform_grid(ψ_0_gpu, qg3p_gpu) ≈ togpu(transform_grid(ψ_0, qg3p_cpu))

Test Passed

In [14]:
@test QG3.SHtoGrid_dμ(ψ_0_gpu, qg3p_gpu) ≈ togpu(QG3.SHtoGrid_dμ(ψ_0, qg3p_cpu))

Test Passed

In [11]:
@test QG3.SHtoGrid_dϕ(ψ_0_gpu, qg3p_gpu) ≈ togpu(QG3.SHtoGrid_dϕ(ψ_0, qg3p_cpu))

Test Passed

In [12]:
@test QG3.SHtoGrid_dλ(ψ_0_gpu, qg3p_gpu) ≈ togpu(QG3.SHtoGrid_dλ(ψ_0, qg3p_cpu))

Test Passed

In [13]:
@test transform_grid(J(ψ_0_gpu, q_0_gpu, qg3p_gpu),qg3p_gpu) ≈ togpu(transform_grid(J(ψ_0, q_0, qg3p_cpu),qg3p_cpu))

Test Passed

In [15]:
A = QG3.QG3MM_gpu(q_0_gpu, [qg3p_gpu, S_gpu], 0.)

B = QG3.QG3MM_base(q_0, [qg3p_cpu, S], 0.)

@test A ≈ QG3.reorder_SH_gpu(B,qg3p_cpu.p)  # time step

Test Passed

In [16]:
DT = T(2π/144)
t_end = T(200.)

# problem definition with GPU model from the library
prob = ODEProblem(QG3.QG3MM_gpu, q_0_gpu, (T(0.),t_end), [qg3p_gpu, S_gpu])

sol = @time solve(prob, AB5(), dt=DT)

@test SciMLBase.successful_retcode(sol)

┌ Warning: Using arrays or dicts to store parameters of different types can hurt performance.
│ Consider using tuples instead.
└ @ SciMLBase ~/.julia/packages/SciMLBase/l4PVV/src/performance_warnings.jl:32


 15.368236 seconds (23.83 M allocations: 1.358 GiB, 5.79% gc time)


Test Passed

transform_fd.jl

In [7]:
using FiniteDifferences, Zygote

In [8]:
g = qg3p_gpu

A = ψ_0_gpu
Ag = Array(transform_grid(ψ_0_gpu, g))
    
# first test the r2r plans gradient correctness 
r2r_plan = QG3.plan_r2r_AD(Ag, 3)
Agf = r2r_plan * Ag

ir2r_plan = QG3.plan_ir2r_AD(Agf, size(Ag,3), 3)

#cpu test
y, back = Zygote.pullback(x -> r2r_plan*x, Ag)
fd_jvp = j′vp(central_fdm(5,1), x -> r2r_plan*x, y, Ag)
diff_val = (fd_jvp[1] - back(y)[1]) 
@test maximum(abs.(diff_val)) < 1e-4

Test Passed

In [12]:
cpudiv = (r2r_plan \ Agf);
gpudiv = (r2r_plan_gpu \ Agf_gpu);
@test cpudiv ≈ Array(gpudiv)

LoadError: UndefVarError: `r2r_plan_gpu` not defined

In [9]:
#cpu test 2 
yi, backi = Zygote.pullback(x -> ir2r_plan*x, Agf)
fd_jvpi = j′vp(central_fdm(5,1), x -> ir2r_plan*x, yi, Agf)
diff_val = (fd_jvpi[1] - backi(yi)[1]) 
@test maximum(abs.(diff_val)) < 1e-3


Test Passed

In [10]:
Ag_gpu = CUDA.CuArray(Ag)

r2r_plan_gpu = QG3.plan_r2r_AD(Ag_gpu, 3)

Agf_gpu = r2r_plan_gpu * Ag_gpu
ir2r_plan_gpu = QG3.plan_ir2r_AD(Agf_gpu, size(Ag_gpu,3), 3)

y_gpu, back_gpu = Zygote.pullback(x -> r2r_plan_gpu*x, Ag_gpu)
diff_val = (fd_jvp[1] - Array(back_gpu(y_gpu)[1])) 
@test maximum(abs.(diff_val)) < 1e-4

Test Passed

In [11]:
gpudiv = Array(ir2r_plan_gpu \ Ag_gpu)
cpudiv = ir2r_plan \ Ag
@test gpudiv[:,:,1:33] ≈ cpudiv[:,:,1:33]

Test Passed

In [12]:
@test gpudiv[:,:,35:end-1] ≈ cpudiv[:,:,end:-1:34]

Test Passed

In [13]:
y_gpu, back_gpu = Zygote.pullback(x -> ir2r_plan_gpu*x, Agf_gpu)

iback_gpu = back_gpu(y_gpu)[1]; 
diff_val = Array(iback_gpu[:,:,1:33]) - fd_jvpi[1][:,:,1:33]
@test maximum(abs.(diff_val)) < 1e-4

Test Passed

In [14]:
diff_val = Array(iback_gpu[:,:,35:end-1]) - fd_jvpi[1][:,:,end:-1:34]
@test maximum(abs.(diff_val)) < 1e-4


Test Passed

In [15]:
AG = transform_grid(ψ_0_gpu, qg3p_gpu);
AG_cpu = Array(AG);

AS = ψ_0_gpu;
AS_cpu = transform_SH(AG_cpu, qg3p_cpu);

In [34]:
y_cpu, back_cpu = Zygote.pullback(x -> transform_grid(x, qg3p_cpu), AS_cpu)
fd_jvp_cpu = j′vp(central_fdm(5,1), x -> transform_grid(x, qg3p_cpu), y_cpu, AS_cpu)
diff = (fd_jvp_cpu[1] - back_cpu(y_cpu)[1])
@test maximum(abs.(diff)) < 1e-4

Test Passed

In [35]:
y_gpu, back_gpu = Zygote.pullback(x -> transform_grid(x, qg3p_gpu), AS);

In [57]:
@test maximum(Array(back_gpu(y_gpu)[1])[:,1:22,1:22] - back_cpu(y_cpu)[1][:,1:22,1:2:end]) < 1e-4

true

In [66]:
@test maximum(back_cpu(y_cpu)[1][:,1:22,2:2:end] - Array(back_gpu(y_gpu)[1])[:,1:22,35:55]) < 1e-5

true

In [22]:
y_cpu, back_cpu = Zygote.pullback(x -> transform_SH(x, qg3p_cpu), AG_cpu)
fd_jvp_cpu = j′vp(central_fdm(5,1), x -> transform_SH(x, qg3p_cpu), y_cpu, AG_cpu)
diff = (fd_jvp_cpu[1] - back_cpu(y_cpu)[1])
@test maximum(abs.(diff)) < 1e-4

Test Passed

In [23]:
y_gpu, back_gpu = Zygote.pullback(x -> transform_SH(x, qg3p_gpu), AG);
@test Array(back_gpu(y_gpu)[1]) ≈ back_cpu(y_cpu)[1]

gpu_cpu_compare.jl

In [67]:
 # or use the function that automatically loads the files that are saved in the repository
    S, qg3ppars, ψ_0, q_0 = QG3.load_precomputed_data()

    # the precomputed fields are loaded on the CPU and are in the wrong SH coefficient convention
    S_gpu, qg3ppars_gpu, ψ_0_gpu, q_0_gpu = QG3.reorder_SH_gpu(S, qg3ppars), togpu(qg3ppars), QG3.reorder_SH_gpu(ψ_0, qg3ppars), QG3.reorder_SH_gpu(q_0, qg3ppars)

    QG3.gpuoff()
    qg3p = CUDA.@allowscalar QG3Model(qg3ppars);
    QG3.gpuon()
    qg3p_gpu = CUDA.@allowscalar QG3Model(qg3ppars_gpu);

    @test QG3.isongpu(qg3p_gpu)
    @test !(QG3.isongpu(qg3p))

    T = eltype(qg3p_gpu)

    a = similar(ψ_0)
    a .= 1

    a_gpu = similar(ψ_0_gpu)
    a_gpu .= 1

    function QG3MM_gpu(q)
        ψ = qprimetoψ(qg3p_gpu, q)
        return - a_gpu .* J(ψ, q, qg3p_gpu) - D(ψ, q, qg3p_gpu) + S_gpu
    end

    function QG3MM_cpu(q)
        ψ = qprimetoψ(qg3p, q)
        return - a .* J(ψ, q, qg3p) - D(ψ, q, qg3p) + S
    end

    g2 = @time gradient(Params([a_gpu])) do
        sum(QG3MM_gpu(q_0_gpu))
    end
    A = g2[a_gpu]

    g = @time gradient(Params([a])) do
        sum(QG3MM_cpu(q_0))
    end
    B = g[a];

    B_gpu = QG3.reorder_SH_gpu(B, qg3ppars);

 53.396992 seconds (36.74 M allocations: 2.193 GiB, 2.89% gc time)
 48.962139 seconds (29.50 M allocations: 1.695 GiB, 2.83% gc time)


In [73]:
@test A ≈ B_gpu 

Test Passed

In [74]:
RELTOL = 1e-5
    RELTOL_PREDICT = 1e-3

    DT = T((2π/144) / 10) # in MM code: 1/144 * 2π
    t_end = T(100.5)

    prob_gpu = ODEProblem(QG3.QG3MM_gpu,q_0_gpu,(T(100.),t_end),[qg3p_gpu, S_gpu])
    sol_gpu = @time solve(prob_gpu, Tsit5(), dt=DT, reltol=RELTOL);

    prob = ODEProblem(QG3.QG3MM_gpu,q_0,(T(100.),t_end),[qg3p, S])
    sol = @time solve(prob, Tsit5(), dt=DT, reltol=RELTOL);

    diff = abs.(QG3.reorder_SH_gpu(sol(t_end),qg3ppars) - sol_gpu(t_end))./sol_gpu(t_end)
    diff[isnan.(diff)] .= 0;



┌ Warning: Using arrays or dicts to store parameters of different types can hurt performance.
│ Consider using tuples instead.
└ @ SciMLBase ~/.julia/packages/SciMLBase/l4PVV/src/performance_warnings.jl:32


 11.599203 seconds (14.96 M allocations: 1020.406 MiB, 7.87% gc time)
  3.968224 seconds (6.20 M allocations: 501.229 MiB, 9.20% gc time)
Test Failed at In[74]:16
  Expression: maximum(diff) < 1.0e-8
   Evaluated: 0.00036528337f0 < 1.0e-8



LoadError: [91mThere was an error during testing[39m

In [77]:
@test maximum(diff) < 1e-3

Test Passed

In [83]:
mean(abs.(diff)) < 1e-6

true